## Linear Regression

In this lab, we will be performing exploratory data analysis and simple linear regression on a data set of housing prices from Ames, Iowa. This is a famous data set both for its size and relative completeness and will serve as an introduction to some off the simpler aspects of data visualization, data cleaning, and the kinds of choices you have to make to get to the point of performing a numerical analysis. 

This lab will start with a brief description of types of data and some practice with data visualization. In the second half, we will perform one variable and multivariable linear regression. 

This lab is based partially on the excellent tutorial by Lee Clemmer: https://www.kaggle.com/leeclemmer/exploratory-data-analysis-of-housing-in-ames-iowa

## Types of data:

#### Sigular Datatypes
* __Booleans__: Variables that take `True` and `False` as values.
* __Integers__: Whole numbers, eg `n = 1232` or `-12`.
* __Floating Point Numbers__: Decimal numbers, eg `f = 12.342` or `1.79e+308`.
* __Strings__: A list of characters, surrounded by `''` or `""`, eg `'This is a string.'`.

#### Types of Collections of Data

* __Lists__: Ordered collections of numbers, strings, floats, of other lists. For example, `[]` is the empty list, `[1,2,3]`, `[[1,2],'Sandwich',4]`. Lists are ordered, with the indexing starting at 0 and counting up.

In [ ]:
y = [[1,2],'Sandwich',4]


* __Dictionaries__: Dictionaries are like lists, but values are stored by a __key__ instead of by an index. A dictionary  is defined by curly braces 

`{'key1': 1, 'key2':[1,2,], 'key3':'Some Text!'}`

In [ ]:
y = {'Color':'Blue', 'Number': 4, 'List': [1,2,3]}


#### Matrices with numpy
Matrix operations can be found in the `numpy` library. Numpy allows easy and efficient matrix operations, as well as some matrix creation mechanism. To create a $2\times 2$ matrix, we use a list `[]` of lists of columns 

`[[col1],[col2],[col3],...]` 

For example, the matrix

$$
y = \left(\begin{matrix}1&2\\3&4\end{matrix}\right)\,,
$$
you use

`y = np.matrix(  [[1,2],[3,4]]  )`

The following are useful matrix commands

* __Transpose__: `A.T`
* __Inverse__: `A.I`
* __Matrix of 0's__: To produce a `n` by `m` matrix of `0`'s, `np.zeros([n,m])`.
* __Matrix of 1's__: To produce a `n` by `m` matrix of `0`'s, `np.ones([n,m])`.
* __Identity Matrix__: For an `n` by `m` identity matrix, `np.eye(n)`.
* __Random Matrix__: A uniform random matrix of size `n` by `m`, can be created with `np.random.random([n,m])`.

In [ ]:
import numpy as np

x = np.random.random([3,2])
y = np.matrix( [[12,3],[1,4]] )



### Displaying output

In the IPython uses two main functions to display output: 

* The python function `print(x)` function prints the raw text of a variable. Print can also be used to construct strings: try setting a value for `n=` and typing `print("There were ", n, "clones chasing me.")` below. Note that Python automatically adds a space between each argument. 

In [ ]:
n = 7


* The IPython function `display(x)` returns the values of `x`, but possibly along with other formatting information. Notice above that if we print the matrix `y` we just get the values, whereas if we `display` it we get some extra information about what type of variable it is.

In [ ]:
display(y)
print(y)

#### Data frames with pandas

A data frame is Python's version of a spreadsheet. It is a matrix where you can label columns and rows and retrieve  information by those labels. Jupyter naturally displays dataframes nicely. A data frame can constructed from a dictionary, where the keys are the column names. Dataframes play particularly nicely with the `display` function.

In [ ]:
import pandas as pd

d = {'Column 1': [1,2,3,4], 'Column 2': [5,6,7,8]}
df = pd.DataFrame(d)

A data frame can also be created from a matrix by specifying the column names when the dataframe is created:

In [ ]:
x = np.random.random([2,2])
df = pd.DataFrame(x, columns=['Column 1','Column 2'])
display(df)

You can access the data from a dataframe by the column name:

In [ ]:
df['Column 1']

## Exploring Data

We will download the Ames, Iowa house pricing data from github. The file itself is a CSV with column headers and so we will use pandas to read it in as a data frame. 

The first step in any data analysis is to understand the data set at a high level, where its coming from and what it indicates:

<div class="alert alert-block alert-info">
DESCRIPTIVE ABSTRACT: Data set contains information from the Ames Assessor’s Office used in computing assessed values for individual residential properties sold in Ames, IA from 2006 to 2010.<br><br>

SOURCES: <br>
Ames, Iowa Assessor’s Office <br><br>

VARIABLE DESCRIPTIONS:<br>
Tab characters are used to separate variables in the data file. The data has 82 columns which include 23 nominal, 23 ordinal, 14 discrete, and 20 continuous variables (and 2 additional observation identifiers). <br><br>

Source: https://www.openintro.org/stat/data/?data=ames<br>
Full Description: http://jse.amstat.org/v19n3/decock/DataDocumentation.txt<br>
</div>

In [ ]:
ames = pd.read_csv('https://raw.githubusercontent.com/hjhuney/Data/master/AmesHousing/train.csv')
ames

### Exploring Datasets:

After reading the high level description, we are ready to dig into the dataset itself. Just displaying the data gave us some information: A brief look at  the content of the columns along with a description of the size, 1460 rows × 81 columns.

There are a few commands particularly useful for first pass data analysis:

* `type(x)` returns the data type of a variable `x`. In this case we know `ames` is a dataframe but this may not always be so clear. 
* `DataFrame.head(n)` a data frame has quite a few built in functions associated with it. The head function returns the first `n` columns of the data frame. To use the head function to display the first 5 rows we use `ames.head(5)`.
* `DataFrame.tail(n)` displays the last `n` columns of the dataframe.
* `DataFrame.shape` is a varaible defining the dimensions of the dataset. You can retrieve them, or change them by changing this variable. 

Lets try these out below

Looking at the output, we make a couple of immediate observations

* Jupyter isn't displaying all 81 of the columns. This is okay, we can get information about them later but it's good to note. 
* We have a large mixture of variable types, some numerical (__Lotarea__) some categorical (__MSZoning__) and others that could be either (__MSSubClass__). You can always check the documentation. 
* The value `NaN` standing for "not a number" keeps appearing. This usually means there's no information about that feature of that particular property. We will have to deal with `NaN` values later. 

We can actually get some information about the data type of each feature and how many values are `NaN` by using the `DataFrame.info()` function. We can also at least get a list of column names using `list(ames)`.

We can immediately read off which columns are numbers and which are other objects. All of this will help us in deciding on which variables its meaningful to use regression to predict the housing price. 

We also see that __PoolQC__, __Fence__, __MiscFeature__ all contain a very small number of non-null objects. A non-null object is any value that isn't `NaN`, so this is saying that we mostly don't have data about __PoolQC__, __Fence__, __MiscFeature__. 

<div class="alert alert-block alert-warning">
<b>Stats Corner:</b> Be warned, being a number does not mean the feature should be treated as numeric. Remember that there are 4 main variable types in statistics:
<ul><li><b>Discrete:</b> varaibles that take a finite number of values with no relation between the values. For example, type of fruit.</li>
    <li><b>Ordinal:</b> variables whose values for a set with a well ordering. For example, stars on a yelp review. 3 is better than 2 but there is no other quantifiable evaluation we can make.  
    <li><b>Interval:</b> continuous variables without a fixed, meaningful 0. For example, temperature in $^\circ C$ can take a continuous value, but $2^\circ C$ isn't twice as hot as $1^\circ C$.
    <li><b>Ratio:</b> continuous variables with a fixed, meaningful 0.
</ul>
Regression might not make sense on discrete variables, and must be more carefully interpreted for ordinal variables. We can however perform regression on binary discrete variables since different choices of order would lead to linear functions differing only by a sign. 
</div>

Finally, lets get some idea of what the target range is. The variable we will try to fit is __SalePrice__ so lets use matplotlib to get an idea of what the data looks like.

#### Plotting Data

We will use `matplotlib` to do most of our plotting. There are some more advanced libraries for producing more complicated graphics objects like maps and network graphs but we will only bring those in when needed. 

We will start my importing `pyplot` from the `matplotlib` library and using the `hist` function to display a histogram of the sale price data.

In [ ]:
from matplotlib import pyplot as plt
fig, ax = plt.subplots()

ax.hist(ames.SalePrice,bins=75, rwidth=.8)
plt.show()

Here, the axis `ax` refers to the specific picture while `fig` refers to an invisible container around it that may contain many pictures inside it. 

This is a good start, but we should always try to make our plots meaning but adding annotation. In this case, lets give the plot a title, label the axes and increase the size of the figure a bit.

* `ax.set_xlabel("X Label")` sets the x axis label for the picture. We can set the fontsize by including `fontsize=`.
* `ax.set_ylabel("Y Label")` sets the y axis label for the picture.
* `ax.set_title("Title")` sets the title for the picture. 
* `fig.set_size_inches(10, 7)` sets the dimensions of the image

In [ ]:
fig, ax = plt.subplots()
fig.set_size_inches(10, 7)

ax.hist(ames.SalePrice,bins=75, rwidth=.8)
plt.show()

Let get the minimum, maximum and average housing cost.

Lets explore some of the other variables, lets take a look at lot size:

In [ ]:
fig, ax = plt.subplots()

ax.hist(ames.LotArea,bins=75, rwidth=.8)
ax.set_xlabel("Lot Size", fontsize=20)
ax.set_ylabel("Count", fontsize=20)
ax.set_title("How big are the lots?", fontsize=20)
fig.set_size_inches(10, 7)
plt.show()

Notice that there are a couple of lots that are so extreme that they're not giving us a very good look at the center. Lets cut out all the value above 50,000 sq ft and plot again. We define a logical index `z=ames.LotArea<50000`, which is just a vector of true and false values. If we put this index into a array of the same size, it will only return that values for which the index `z` is true:

In [ ]:
fig, ax = plt.subplots()


## Scatter Plots for Numerical Features

In this lab, we really want to look at the numerical features, and in fact only the ordinal features. To pull out the values we will use another built in function of dataframes, the `select_dtype` function:

* `DataFrame.select_dtypes(include=['dtype1','dtype2'])` returns a dataframe that only includes the datatypes `'dtype1'`, `'dtype2'`, etc.

In this case we want to extract the numerical columns, so we will include `int64` and `float64`.

For an pair of features, we can form the scatter plot with `matplotlib`'s `plt.plot` function. Without any extra parameters, `plt.plot` returns a line plot, but we can set it to a scatter plot by including a format string, in this case `'o'` (lowercase o). 

* `ax.plot(x,y,'o')` Plots points whose $x$ coordinates are given in the first vector and whose $y$ coordinates are given in the second vector. The format string `'o'` indicates a scatter plot. 

To see some other options for formatting, take a look through matplotlibs own tutorial here: https://matplotlib.org/tutorials/introductory/pyplot.html

In [ ]:
fig, ax = plt.subplots()
fig.set_size_inches(10, 7)

ax.plot(nums.LotArea,nums.SalePrice,'o')
ax.set_xlabel("Lot Size", fontsize=20)
ax.set_ylabel("Sale Price", fontsize=20)
ax.set_title("Lots vs Sales", fontsize=20)
plt.show()

Again, we can cut out the outliers using the logical index `z` from before.

In [ ]:
fig, ax = plt.subplots()


And in fact we may just want to excise them all together:

In [ ]:
numsc = nums[z]   ## Here, the c stands for "cleaned"

Lets compare some of the other data sets:

In [ ]:
fig, ax = plt.subplots()


#### Subplots

To compare multiple features at once it's useful to create a matrix of side by side subplots. To create such a matrix, we edit the subplots call to include a shape vector describing the number of subplots we want

* `fig, axes = plt.subpolots(m,n)` returns an `m` by `n` matrix of subplots. It stores the reference varisble for each subplot in the matrix of subplots in the list `axes`. It also returns a reference to the overall figure containing the subplots, which we store in `fig`. You can plot to the subplot in position (i,j) with `axes[i,j].plot(xvalues, yvalues)`.

We can also specify the names inside the call to each plot

In [ ]:
fig, axes = plt.subplots(2,2, sharey=True)
fig.set_size_inches(10, 10)

#axes[0,0].plot(numsc.GarageArea,numsc.SalePrice,'o')
#axes[1,0].plot(numsc.LotArea,numsc.SalePrice,'o')
#axes[0,1].plot(numsc.OverallCond,numsc.SalePrice,'o')
#axes[1,1].plot(numsc.YrSold,numsc.SalePrice,'o')

plt.show()

We would like to see all scatterplots for all of the variables, properly named. To achive this we will loop through all of the data using a for loop. Using `list(numsc)` and the `len` function we can get the lenth of the list of column names for the numeric columns. 

We see that there are 38 columns containing numeric data, one of which contains the sales price. This can be roughly a $4\times 10$ grid. 

One way to display all of the scatter plots is to loop through the axes using two for loops, one running over the columns `m` and one over the rows `n`. The other way is to flatten the 2d array into a list of axes handlers and just loop through the list. In this case, we'd flatten our $4\times 10$ grid into a single vector of length 40 using `.reshape()`. For example,

In [ ]:
A =  np.matrix([[0, 1],
                [2, 3],
                [4, 5],
                [6, 7]])

print(A.reshape())
print(A.reshape())

For the grid of scatter plots we start with a 4 by 10 array, reshape it a length 40 array, and finally loop through it, plotting the scatter plots as we go. We do have to call the scatter plots by name, so we will actually use the `i`'th index of the `nnames` vector. 

We also want to label each axis, we can do this with 

`axes[i].set_title(nnames[i], fontsize=16)`

In [ ]:
f, axes = plt.subplots(10, 4, sharey = True)
f.set_size_inches(15,30)
f.tight_layout()


To clean this display up a bit, we can drop `id` from the list of numeric variables as it appears to be essentially random. 

A quick glance immediately tells us which variables should be treated as discrete or ordinal (which we have to be more careful about fitting with linear regression) and which should be treated as interval or ratio (which is linear regression's natural setting). 

There are many type of plots we could produce this way. For exploratory data analysis another type of plot could include frequency data.

In [ ]:
f, axes = plt.subplots(2, 4)
f.set_size_inches(15,8)
f.tight_layout()

axes = axes.reshape(8)



## Correlation Plot

As a last bit of exploratory analysis before moving on to prediction we will produce and plot the correlation matrix for the numerical features. To do so we will use Seaborn, a library for statistical data visualization built on top of `matplotlib`. There is of course the usual debate in the community about whether you should learn seaborn first, since it's easier and more elegant or if you should learn matplotlib first, since it's more fundamental. 

* `DataFrame.corr()` is a built in function that compute the correlation between the columns of a dataframe. 

We will use seaborns `sns.heatmap` function to produce a plot of all of the correlations. Seaborn often does an excellent job of displaying things on its own, but it is also deeply customization. In this case, we'll add a `linewidths=0.05` argument to make the correlation more distinct and use the `cmap="magma"` option to get a slightly more readable color scheme. 

In [ ]:
import seaborn as sns
fig,ax = plt.subplots(figsize=(14, 10))



We see that __OverallQual__, __TotalBsmtSF__, __1stFlrSF__, __GarageArea__, __GrLivArea__, are all highly correlated with sales price. So high value houses have big garages, and large square footage and are overall of good quality.

## Linear Regression

We now want to perform our first predictions using linear regression. We will do a quick one variable fit both using linear algebra and then using the sci-kit learn library and compare the fits. 

For a single feature, we want to fit a 2 vector of constants $[\beta_0,\beta_1]$ to 
$$
Y = \beta_0 + X\beta_1 = {X}^T\beta = [1,X] \left[ \begin{matrix} \beta_0\\\beta_1 \end{matrix} \right] \,.
$$
The solution will be given by
$$
\beta = (\mathbf{X}^T\mathbf{X})^{-1}\mathbf{X}^T\mathbf{y}\,,
$$
where $\mathbf{X}$ is the vector where each column is $[1,x_i]$ contains the $x_i$'s measured feature and $\mathbf{y}$ is a vector of target values.

Lets start by splitting the data into training data and test data. For now, we'll just save the last 200 values as test data. 

In [ ]:
#X_train = numsc['1stFlrSF'][0:-200]
#X_test = numsc['1stFlrSF'][-200:]


Next, we have to append a column of a's to the X vector to account for the intercept term $\beta_0$. This also have the effect ensuring that $\mathbf{X}^T\mathbf{X}$ is actually invertable. On way to do this is to just concatinate the vectors and then reshape them into the proper form. 

In [ ]:
#X = np.append(np.ones(1249), X_train, 0)
#X = X.reshape(2,1249).T

We now change `X` and `Y` to matricies and perform compute the regression minimum.

Lets plot the resulting trend line:

Lets check how we did on our test set, both with a graph and numerically. Graphically, we can just plot the regression line again the test data scatter plot. We can evaluate the model numerically but computing the residual sum squared
$$
RSS(\beta) = \sum_{i=1}^N (y_i - x_i^T\beta)^2\,,
$$
the root mean square
$$
RMS(\beta) = \left(\frac{1}{N}(y_i - x_i^T\beta)^2\right)^{\frac12}\,,
$$
and the $r^2$ value
$$
r^2 = 1 - \frac{RSS(\beta)}{\sum_{i=1}^N (y_i - E[y])^2}\,.
$$

Now lets use sci-kit learns built in regression function. We import `LinearRegression` from `sklearn.linear_model`, the toolkit of linear models in sci-kit learn. We then set up a linear regression object using

`lr = LinearRegression()`

An object like a linear regression object is a structure like a dataframe. It stores data, but also has a series of utility function associated with it. It is a self contained machine that we can put data into, turn a crank (by calling functions) and have it process and return that data. In this case, it takes in training data and fits a linear regressor to it. We can then ask it return the parameters of that regressor with 

* `lr.coef_` Returns the slope coefficients of regression $\beta_1,\ldots, \beta_p$.
* `lr.intercept_` Returns the intercept of regression $\beta_0$.

Note, that `LinearRegression` wants the data in the form we've been using in class: the $X$ data should be a $N$ by $p$ matrix of data points and the $y$ data should be a $N$ by 1 column vector. 

In [ ]:
from sklearn.linear_model import LinearRegression
lr = LinearRegression()


The `LinearRegression` object contains a in built $r^2$ scoring function: 

* `LinearRegression.score(X_data,Y_data)` takes an array of input data `X_data` an array of label data `Y_data` and returns the $r^2$ score of the regressor on the data. 

In [ ]:
lr.score(np.array(X_test).reshape(-1,1),np.array(Y_test).reshape(-1,1))

#### Regression with statsmodels.api

Although in this class we will be focusing on sci-kit learn, there's another important statistics library to be aware of, statsmodels.api. For regression in particular, statsmodels.api is just a much better tool, performing all of the statistical analysis you would like. The only caveat is that stats model doesn't naturally fit to a constant term, so we must again add a column of 1's. But statsmodels.api has a function to preform that task.

In [ ]:
import statsmodels.api as sm
x_train_SM = X_train
x_train_SM = sm.add_constant(x_train_SM)
ols = sm.OLS(Y_train, x_train_SM)
ols_result = ols.fit()
# Now you have at your disposition several error estimates, e.g.
ols_result.summary()

## Multilinear Regression

It actually often takes less computational time to compute a linear regression than it does to plot it. We want to fit the set of training data with all the numerical features to the sale price, but before we do we have to deal with those `NaN` values. For example, if we just try to compute the linear regression now it will return an error. 

In [ ]:
X_train = numsc[0:-200]
X_test = numsc[-200:]

lr.fit(np.array(X_train).reshape(-1,1),np.array(Y_train).reshape(-1,1))

Fixing missing values is fiddly, and should be done with care. Whatever choice you make will effect your fit, and as a result will effect your accuracy. In general, data cleaning is the usually more than half the battle. 

To find the `NaN` values, we will use the `DataFrame.isnull().sum()$ to return all of the null values, and them sum them along columns. 

We find that __LotFrontage__, __MasVnrArea__, and __GarageYrBlt__ are the only numerical features with null values. Lets take a look at each of these values individually. After we make a choice for each variable, we will use

* `DataFrame.Feature.fillna(Value, inplace=True)` fills all of the `NaN` values in an array with the contents of `Value`. 

__MasVnrArea__ is "Masonry veneer area in square feet". Dumping the data we find a lot of zero values, so its probably safe to assume that 8 values can be set to 0 or the median value. 

For __GarageYrBlt__, a value of `NaN` probably means there's no garage. Lets check just to make sure by looking at the __GarageArea__ variable for every `NaN` value.

Finally, for __LotFrontage__ we assume that almost every lot has some frontage road so we don't want to set this to 0. There are more sophisticated solutions, but the simplest is just to find the median value and set the null values to it. 

Great, so we have fixed all of the `NaN` values. Lets compute the linear fit.

Lets see what the parameters are:

In [ ]:
beta = np.insert(lr.coef_.reshape(1,37),0, lr.intercept_, axis=1)
pd.DataFrame(beta,columns=["Const"]+list(X_train))

## Exercise:

Find the single variable linear predictor which gives the lowest error on the test set. You must justify your answer.